In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import json
from datetime import datetime

import requests
from bs4 import BeautifulSoup

In [23]:
caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

options = Options()

options.add_experimental_option("detach", True)

options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome(options=options)

url = 'https://www.google.com/finance/?hl=ja'
driver.get(url)

time.sleep(1)


In [24]:
stock_code_input = driver.find_element(By.XPATH, "/html/body/c-wiz[2]/div/div[3]/div[3]/div/div/div/div[1]/input[2]")
stock_code_input.send_keys("NVDA")
stock_code_input.send_keys(Keys.RETURN)

In [13]:
driver.find_element(By.XPATH, r'//*[@id="6monthTab"]').click()

logs = driver.get_log('performance')

In [25]:
current_url = driver.current_url
new_url = f"{current_url}&window=6M"
driver.get(new_url)
logs = driver.get_log('performance')

In [27]:
print(logs)

[{'level': 'INFO', 'message': '{"message":{"method":"Page.frameStartedLoading","params":{"frameId":"47823143AC24A4E92FF3B8607644C3CF"}},"webview":"47823143AC24A4E92FF3B8607644C3CF"}', 'timestamp': 1721973794089}, {'level': 'INFO', 'message': '{"message":{"method":"Network.requestWillBeSent","params":{"documentURL":"https://www.google.com/finance/?hl=ja","frameId":"47823143AC24A4E92FF3B8607644C3CF","hasUserGesture":false,"initiator":{"type":"other"},"loaderId":"A1C4A5DFF7AC8F8A6C362B68F5BD63D0","redirectHasExtraInfo":false,"request":{"headers":{"Upgrade-Insecure-Requests":"1","User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36","sec-ch-ua":"\\"Not/A)Brand\\";v=\\"8\\", \\"Chromium\\";v=\\"126\\", \\"Google Chrome\\";v=\\"126\\"","sec-ch-ua-mobile":"?0","sec-ch-ua-platform":"\\"Windows\\""},"initialPriority":"VeryHigh","isSameSite":true,"method":"GET","mixedContentType":"none","referrerPolicy":"strict-origin-wh

In [26]:
batch_request_url = None
headers = {}
post_data = None


for entry in logs:
    log = json.loads(entry['message'])['message']
    try:
        if 'Network.requestWillBeSent' in log['method']:
            request_url = log['params']['request']['url']
            if 'https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=' in request_url:
                batch_request_url = request_url
                headers = log['params']['request']['headers']
                if 'postData' in log['params']['request']:
                    post_data = log['params']['request']['postData']
                break
    except KeyError:
        continue

# driver.quit()
print(batch_request_url)

https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=mKsvE&source-path=%2Ffinance%2F&f.sid=-3141341818194722963&bl=boq_finance-ui_20240610.00_p0&hl=ja&soc-app=162&soc-platform=1&soc-device=1&_reqid=154199&rt=c


In [21]:
if batch_request_url:
    print(batch_request_url)
    if post_data:
        response = requests.post(batch_request_url, headers=headers, data=post_data)
    else:
        response = requests.get(batch_request_url, headers=headers)

    print("Status Code:", response.status_code)
    print("Response Text:", response.text)
    try:
        data = response.json()
        print(data)
    except json.JSONDecodeError:
        print("レスポンスのJSONデコードに失敗しました")
else:
    print("バッチリクエストURLが見つかりませんでした")

https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=mKsvE&source-path=%2Ffinance%2F&f.sid=1392060461472508506&bl=boq_finance-ui_20240610.00_p0&hl=ja&soc-app=162&soc-platform=1&soc-device=1&_reqid=154104&rt=c
Status Code: 200
Response Text: )]}'

3549
[["wrb.fr","mKsvE","[[[\"/m/07zllzd\",null,null,[\"/m/07zllzd\",[\"NVDA\",\"NASDAQ\"],\"NVIDIA\",0,\"USD\",[112.28,-1.9700012,-1.7242899,2,2,2],null,114.25,\"#5b9000\",\"US\",\"/m/09rh_\",[1721968261],\"America/New_York\",-14400,\"/m/07zllzd\",null,[112.24,-0.040000916,-0.03562604,2,3,3],[1721937601],[1721951998],[[1,[2024,7,25,9,30,null,null,[-14400]],[2024,7,25,16,null,null,null,[-14400]]]],null,\"NVDA:NASDAQ\",0],[null,[\"NVDA\",\"NASDAQ\"]]],[\"/g/1q62j_hn9\",null,null,[\"/g/1q62j_hn9\",[\"NVDA\",\"BMV\"],\"NVIDIA\",0,\"MXN\",[2070.03,-34.629883,-1.6453909,2,2,2],null,2104.66,\"#5b9000\",\"MX\",\"/m/09rh_\",[1721939012],\"America/Mexico_City\",-21600,\"/g/1q62j_hn9\",null,null,[1721937600],null,[[1,[2024,7,25,8,30

In [16]:
def remove_before_bracket_and_after_newline(s):
    # '['前の文字を削除
    bracket_index = s.find('[')
    if bracket_index != -1:
        s = s[bracket_index:]

    # 改行以降の文字を削除
    newline_index = s.find('\n')
    if newline_index != -1:
        s = s[:newline_index]

    return s

def parse_array_string(array_string):
    def parse_helper(s):
        # 空白文字を除去
        s = s.strip()
        if s.startswith('[') and s.endswith(']'):
            s = s[1:-1]
        else:
            return s.strip()  # Base case: return the stripped string itself

        result = []
        nested_level = 0
        start_index = 0

        for i, char in enumerate(s):
            if char == '[':
                if nested_level == 0:
                    start_index = i
                nested_level += 1
            elif char == ']':
                nested_level -= 1
                if nested_level == 0:
                    result.append(parse_helper(s[start_index:i+1]))
            elif char == ',' and nested_level == 0:
                result.append(parse_helper(s[start_index:i]))
                start_index = i + 1

        if start_index < len(s):
            result.append(parse_helper(s[start_index:]))

        return result

    return parse_helper(array_string)


res_text = remove_before_bracket_and_after_newline(response.text)

parsed_array = parse_array_string(res_text)

# print(res_text)
print(type(res_text))
print(type(parsed_array))
print(len(parsed_array[0][2][0][0][4][0][2]))
print(parsed_array[0][2][0][0][4][0][2][0])
print(parsed_array[0][2][0][0][4][0][2][0][0])
print(parsed_array[0][2][0][0][4][0][2][0][0][0])
print(parsed_array[0][2][0][0][4][0][2][0][0][1])
print(parsed_array[0][2][0][0][4][0][2][0][0][2])
print(parsed_array[0][2][0][0][4][0][2][0][2][0])

<class 'str'>
<class 'list'>
1
2
2
2


IndexError: string index out of range

In [ ]:
for data_array in parsed_array[0][2][0][0][4][0][2]:
    print(data_array[0][0], data_array[0][1], data_array[0][2], data_array[2][0])

2024 1 23 59.87
2024 1 23 59.87
2024 1 24 61.36
2024 1 24 61.36
2024 1 25 61.62
2024 1 25 61.62
2024 1 26 61.03
2024 1 26 61.03
2024 1 29 62.47
2024 1 29 62.47
2024 1 30 62.77
2024 1 30 62.77
2024 1 31 61.53
2024 1 31 61.53
2024 2 1 63.03
2024 2 1 63.03
2024 2 2 66.16
2024 2 2 66.16
2024 2 5 69.33
2024 2 5 69.33
2024 2 6 68.22
2024 2 6 68.22
2024 2 7 70.1
2024 2 7 70.1
2024 2 8 69.64
2024 2 8 69.64
2024 2 9 72.13
2024 2 9 72.13
2024 2 12 72.25
2024 2 12 72.25
2024 2 13 72.13
2024 2 13 72.13
2024 2 14 73.9
2024 2 14 73.9
2024 2 15 72.66
2024 2 15 72.66
2024 2 16 72.61
2024 2 16 72.61
2024 2 20 69.45
2024 2 20 69.45
2024 2 21 67.47
2024 2 21 67.47
2024 2 22 78.54
2024 2 22 78.54
2024 2 23 78.82
2024 2 23 78.82
2024 2 26 79.09
2024 2 26 79.09
2024 2 27 78.7
2024 2 27 78.7
2024 2 28 77.66
2024 2 28 77.66
2024 2 29 79.11
2024 2 29 79.11
2024 3 1 82.28
2024 3 1 82.28
2024 3 4 85.24
2024 3 4 85.24
2024 3 5 85.96
2024 3 5 85.96
2024 3 6 88.7
2024 3 6 88.7
2024 3 7 92.67
2024 3 7 92.67
2024 3 8

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import json
from datetime import datetime

import requests

caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

options = Options()

options.add_experimental_option("detach", True)

options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome(options=options)

with open('./scrape_data/stock_name.json', 'r', encoding="utf-8") as file:
    data = json.load(file)

url = "https://www.google.com/finance/?hl=ja"
driver.get(url)

time.sleep(1)

logs = driver.get_log('performance')

stock_code_input = driver.find_element(By.XPATH, "/html/body/c-wiz[2]/div/div[3]/div[3]/div/div/div/div[1]/input[2]")
stock_code_input.send_keys("NASDAQ")
stock_code_input.send_keys(Keys.RETURN)
time.sleep(1)


for category in data:
    for company in data[category]:
        stock_code = company["symbol"]
        print(stock_code)

        stock_code_cross = driver.find_element(By.XPATH, r'//*[@id="gb"]/div[2]/div[2]/div/form/button[2]').click()

        stock_code_reflesh = driver.find_element(By.XPATH, r'//*[@id="gb"]/div[2]/div[2]/div/form/div/div/span/div/div/div/div[1]/input[2]')
        # stock_code_reflesh.clear()
        time.sleep(1)

        stock_code_reflesh.send_keys(stock_code)
        stock_code_reflesh.send_keys(Keys.RETURN)
        time.sleep(2)

        current_url = str(driver.current_url)
        print(current_url)
        new_url = f"{current_url}&window=6M"
        driver.get(new_url)
        logs = driver.get_log('performance')
        time.sleep(2)

        batch_request_url = None
        headers = {}
        post_data = None

        for entry in logs:
            log = json.loads(entry['message'])['message']
            try:
                if 'Network.requestWillBeSent' in log['method']:
                    request_url = log['params']['request']['url']
                    if 'https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=' in request_url and stock_code in request_url:
                        batch_request_url = request_url
                        headers = log['params']['request']['headers']
                        if 'postData' in log['params']['request']:
                            post_data = log['params']['request']['postData']
                        break
            except KeyError:
                continue

        if batch_request_url:
            print(batch_request_url)
            if post_data:
                response = requests.post(batch_request_url, headers=headers, data=post_data)
            else:
                response = requests.get(batch_request_url, headers=headers)

            print("Status Code:", response.status_code)
            print("Response Text:", response.text)
            # try:
            #     data = response.json()
            #     print(data)
            # except json.JSONDecodeError:
            #     print("レスポンスのJSONデコードに失敗しました")
        else:
            print("バッチリクエストURLが見つかりませんでした")


        def remove_before_bracket_and_after_newline(s):
            # '['前の文字を削除
            bracket_index = s.find('[')
            if bracket_index != -1:
                s = s[bracket_index:]

            # 改行以降の文字を削除
            newline_index = s.find('\n')
            if newline_index != -1:
                s = s[:newline_index]

            return s


        def parse_array_string(array_string):
            def parse_helper(s):
                # 空白文字を除去
                s = s.strip()
                if s.startswith('[') and s.endswith(']'):
                    s = s[1:-1]
                else:
                    return s.strip()  # Base case: return the stripped string itself

                result = []
                nested_level = 0
                start_index = 0

                for i, char in enumerate(s):
                    if char == '[':
                        if nested_level == 0:
                            start_index = i
                        nested_level += 1
                    elif char == ']':
                        nested_level -= 1
                        if nested_level == 0:
                            result.append(parse_helper(s[start_index:i+1]))
                    elif char == ',' and nested_level == 0:
                        result.append(parse_helper(s[start_index:i]))
                        start_index = i + 1

                if start_index < len(s):
                    result.append(parse_helper(s[start_index:]))

                return result

            return parse_helper(array_string)


        res_text = remove_before_bracket_and_after_newline(response.text)

        parsed_array = parse_array_string(res_text)

        for data_array in parsed_array[0][2][0][0][4][0][2]:
            print(data_array[0][0], data_array[0][1], data_array[0][2], data_array[2][0])

A
https://www.google.com/finance/quote/A:NYSE?hl=ja
https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=HqGpWd%2Cxh8wxf%2Co6pODe%2CnBEQBc%2CmKsvE%2CuwlMvd%2CSICF5d%2CAiCwsd%2CPr8h2e%2CyYvDpf&source-path=%2Ffinance%2Fquote%2FNDX%3AINDEXNASDAQ&f.sid=-843396336099856088&bl=boq_finance-ui_20240610.00_p0&hl=ja&soc-app=162&soc-platform=1&soc-device=1&_reqid=149257&rt=c
Status Code: 200
Response Text: )]}'

48
[["wrb.fr","yYvDpf","[]",null,null,null,"29"]]
595
[["wrb.fr","HqGpWd","[[[\"/m/02fhwv\",\"ナスダック100指数\",\"ナスダック100指数は、ナスダックに上場する、金融銘柄を除く、時価総額上位100銘柄の時価総額加重平均によって算出される株価指数である。アメリカ合衆国を代表する株価指数の一つとなっている。\",null,null,null,null,null,19032.389,19041.152,19227.2,18721.705,20690.967,14058.333,1393614291,null,null,null,null,null,null,null,null,null,null,0,0,0,1,null,\"https://ja.wikipedia.org/wiki/%E3%83%8A%E3%82%B9%E3%83%80%E3%83%83%E3%82%AF100%E6%8C%87%E6%95%B0\",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2,2,2,2,2,0,0,null,null,null,null,n

IndexError: list index out of range